In [1]:
!pip install matplotlib


In [2]:
import tkinter as tk
from tkinter import ttk, messagebox, filedialog
import math
from scipy.stats import norm
import matplotlib.pyplot as plt
import csv

# =================== FINANCIAL FUNCTIONS ======================

def black_scholes_price(S, K, T, r, sigma, option_type='call'):
    d1 = (math.log(S / K) + (r + 0.5 * sigma**2) * T) / (sigma * math.sqrt(T))
    d2 = d1 - sigma * math.sqrt(T)

    if option_type == 'call':
        price = S * norm.cdf(d1) - K * math.exp(-r * T) * norm.cdf(d2)
    elif option_type == 'put':
        price = K * math.exp(-r * T) * norm.cdf(-d2) - S * norm.cdf(-d1)
    else:
        raise ValueError("Invalid option type")
    
    return round(price, 4)

def calculate_greeks(S, K, T, r, sigma, option_type='call'):
    d1 = (math.log(S / K) + (r + 0.5 * sigma**2) * T) / (sigma * math.sqrt(T))
    d2 = d1 - sigma * math.sqrt(T)
    
    delta = norm.cdf(d1) if option_type == 'call' else norm.cdf(d1) - 1
    gamma = norm.pdf(d1) / (S * sigma * math.sqrt(T))
    vega = S * norm.pdf(d1) * math.sqrt(T) / 100
    theta_call = (-S * norm.pdf(d1) * sigma / (2 * math.sqrt(T))
                  - r * K * math.exp(-r * T) * norm.cdf(d2)) / 365
    theta_put = (-S * norm.pdf(d1) * sigma / (2 * math.sqrt(T))
                 + r * K * math.exp(-r * T) * norm.cdf(-d2)) / 365
    rho_call = K * T * math.exp(-r * T) * norm.cdf(d2) / 100
    rho_put = -K * T * math.exp(-r * T) * norm.cdf(-d2) / 100

    theta = theta_call if option_type == 'call' else theta_put
    rho = rho_call if option_type == 'call' else rho_put
    
    return {
        'Delta': round(delta, 4),
        'Gamma': round(gamma, 4),
        'Vega': round(vega, 4),
        'Theta': round(theta, 4),
        'Rho': round(rho, 4)
    }

# =================== GUI SETUP ======================

def launch_gui():
    def calculate_all():
        try:
            S = float(entry_S.get())
            K = float(entry_K.get())
            T = float(entry_T.get())
            r = float(entry_r.get())
            sigma = float(entry_sigma.get())
            option_type = option_var.get()

            price = black_scholes_price(S, K, T, r, sigma, option_type)
            greeks = calculate_greeks(S, K, T, r, sigma, option_type)

            result_text.set(f"{option_type.capitalize()} Price: {price}")
            for g in greek_labels:
                greek_labels[g].config(text=f"{g}: {greeks[g]}")
            # Save for export
            last_result.clear()
            last_result.update({'S': S, 'K': K, 'T': T, 'r': r, 'sigma': sigma,
                                'type': option_type, 'price': price, **greeks})
        except Exception as e:
            messagebox.showerror("Error", str(e))

    def export_csv():
        if not last_result:
            messagebox.showwarning("No data", "Please calculate first.")
            return
        file_path = filedialog.asksaveasfilename(defaultextension=".csv")
        if file_path:
            with open(file_path, mode='w', newline='') as file:
                writer = csv.writer(file)
                writer.writerow(last_result.keys())
                writer.writerow(last_result.values())
            messagebox.showinfo("Saved", f"Results saved to {file_path}")

    def plot_price_vs_volatility():
        try:
            S = float(entry_S.get())
            K = float(entry_K.get())
            T = float(entry_T.get())
            r = float(entry_r.get())
            option_type = option_var.get()

            sigmas = [i / 100 for i in range(5, 105, 5)]
            prices = [black_scholes_price(S, K, T, r, sigma, option_type) for sigma in sigmas]

            plt.figure(figsize=(6,4))
            plt.plot([s*100 for s in sigmas], prices, marker='o')
            plt.xlabel("Volatility (%)")
            plt.ylabel("Option Price")
            plt.title("Option Price vs Volatility")
            plt.grid(True)
            plt.show()
        except Exception as e:
            messagebox.showerror("Error", str(e))

    def plot_price_vs_time():
        try:
            S = float(entry_S.get())
            K = float(entry_K.get())
            sigma = float(entry_sigma.get())
            r = float(entry_r.get())
            option_type = option_var.get()

            times = [i/12 for i in range(1, 25)]
            prices = [black_scholes_price(S, K, T, r, sigma, option_type) for T in times]

            plt.figure(figsize=(6,4))
            plt.plot(times, prices, marker='o')
            plt.xlabel("Time to Maturity (Years)")
            plt.ylabel("Option Price")
            plt.title("Option Price vs Time")
            plt.grid(True)
            plt.show()
        except Exception as e:
            messagebox.showerror("Error", str(e))

    # Memory for export
    last_result = {}

    # === Main Window ===
    root = tk.Tk()
    root.title("Black-Scholes Option Pricing Calculator")

    tab_control = ttk.Notebook(root)

    # === Tab 1: Input & Results ===
    tab_input = ttk.Frame(tab_control)
    tab_control.add(tab_input, text="Inputs & Results")

    labels = ["Stock Price (S)", "Strike Price (K)", "Time to Maturity (T)", 
              "Risk-Free Rate (r)", "Volatility (σ)"]
    entries = []
    for i, label in enumerate(labels):
        tk.Label(tab_input, text=label).grid(row=i, column=0, sticky='e')
        entry = tk.Entry(tab_input)
        entry.grid(row=i, column=1)
        entries.append(entry)

    entry_S, entry_K, entry_T, entry_r, entry_sigma = entries

    tk.Label(tab_input, text="Option Type").grid(row=5, column=0, sticky='e')
    option_var = tk.StringVar(value="call")
    option_menu = ttk.Combobox(tab_input, textvariable=option_var, values=["call", "put"], state="readonly")
    option_menu.grid(row=5, column=1)

    tk.Button(tab_input, text="Calculate", command=calculate_all).grid(row=6, column=0, columnspan=2, pady=10)

    result_text = tk.StringVar()
    tk.Label(tab_input, textvariable=result_text, font=("Helvetica", 12), fg="blue").grid(row=7, column=0, columnspan=2)

    greek_labels = {}
    for i, greek in enumerate(["Delta", "Gamma", "Vega", "Theta", "Rho"]):
        greek_labels[greek] = tk.Label(tab_input, text=f"{greek}: ")
        greek_labels[greek].grid(row=8+i, column=0, columnspan=2)

    # === Tab 2: Plots ===
    tab_plot = ttk.Frame(tab_control)
    tab_control.add(tab_plot, text="Plots")

    tk.Button(tab_plot, text="Plot Price vs Volatility", command=plot_price_vs_volatility).pack(pady=10)
    tk.Button(tab_plot, text="Plot Price vs Time", command=plot_price_vs_time).pack(pady=10)

    # === Tab 3: Export ===
    tab_export = ttk.Frame(tab_control)
    tab_control.add(tab_export, text="Export")

    tk.Button(tab_export, text="Export Last Result to CSV", command=export_csv).pack(pady=20)

    # === Tab 4: About ===
    tab_about = ttk.Frame(tab_control)
    tab_control.add(tab_about, text="About")

    tk.Label(tab_about, text="Black-Scholes Option Pricing Calculator", font=("Arial", 12, "bold")).pack(pady=10)
    tk.Label(tab_about, text="Built by: EulerOnWallSt").pack()
    tk.Label(tab_about, text="Includes price, Greeks, export, and plotting").pack()
    tk.Label(tab_about, text="Not financial advice. Use responsibly.").pack(pady=20)

    tab_control.pack(expand=1, fill="both")
    root.mainloop()

# Launch GUI
launch_gui()
